In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib sentencepiece
!pip install tensorboardX
!pip install -q git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/peft.git

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training

import os
import torch
import torch.nn as nn

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll
CUDA SETUP: CUDA runtime path found: C:\Users\panta\anaconda3\envs\nlp\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll...


c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: C:\Users\panta\anaconda3\envs\nlp did not contain ['cudart64_110.dll', 'cudart64_120.dll'] as expected! Searching further paths...
  warn(msg)
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/usr/bin'), WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/mingw-w64/bin')}
  warn(msg)


In [2]:
BASE_MODEL = "facebook/opt-1.3b"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = 0

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\panta\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 3145728 || all params: 1318903808 || trainable%: 0.23851079820371554


In [4]:
def tokenize_sample(item, max_seq_length=1024, add_eos_token=True):
        result = tokenizer(
            item,
            truncation=True,
            max_length=max_seq_length,
            padding=True,
        )
        result = {
            "input_ids": result["input_ids"][:-1],
            "attention_mask": result["attention_mask"][:-1],
        }
        if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < max_seq_length
            and add_eos_token
        ):
            result["input_ids"].append(tokenizer.eos_token_id)
            result["attention_mask"].append(1)
        
        return result

In [5]:
def generate_prompt(data_point):
    return f"### Caption: {data_point['caption_string']}\n### Tags:{data_point['tag_string']}"

In [8]:
from datasets import load_dataset

data = load_dataset("json", data_files=r'dataset/train_data.json')
data = data["train"].train_test_split(test_size=0.05, shuffle=True, seed=42)
data = data.map(lambda x: tokenize_sample(generate_prompt(x)))
data

Found cached dataset json (C:/Users/panta/.cache/huggingface/datasets/json/default-fac367448397b4f6/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/18952 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'tag_string', 'input_ids', 'attention_mask'],
        num_rows: 18952
    })
    test: Dataset({
        features: ['caption_string', 'tag_string', 'input_ids', 'attention_mask'],
        num_rows: 998
    })
})

In [10]:
trainer = transformers.Trainer(
    model=model, 
    train_dataset=data['train'],
    eval_dataset=data['test'],
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=32,
        num_train_epochs=3,
        learning_rate=1e-4, 
        fp16=True,
        logging_steps=2,
        optim="adamw_torch",
        evaluation_strategy="steps",
        eval_steps=20,
        output_dir='outputs',
#         save_strategy="steps",
#         save_steps=20,
#         save_total_limit=2,
        report_to="tensorboard"
    ),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

  0%|          | 0/444 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\autograd\_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'loss': 2.947, 'learning_rate': 9.954954954954956e-05, 'epoch': 0.01}
{'loss': 2.8607, 'learning_rate': 9.90990990990991e-05, 'epoch': 0.03}
{'loss': 2.7888, 'learning_rate': 9.864864864864865e-05, 'epoch': 0.04}
{'loss': 2.7105, 'learning_rate': 9.81981981981982e-05, 'epoch': 0.05}
{'loss': 2.6408, 'learning_rate': 9.774774774774775e-05, 'epoch': 0.07}
{'loss': 2.6105, 'learning_rate': 9.729729729729731e-05, 'epoch': 0.08}
{'loss': 2.5864, 'learning_rate': 9.684684684684685e-05, 'epoch': 0.09}
{'loss': 2.5305, 'learning_rate': 9.639639639639641e-05, 'epoch': 0.11}
{'loss': 2.497, 'learning_rate': 9.594594594594595e-05, 'epoch': 0.12}
{'loss': 2.4375, 'learning_rate': 9.54954954954955e-05, 'epoch': 0.14}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.3455841541290283, 'eval_runtime': 104.7633, 'eval_samples_per_second': 9.526, 'eval_steps_per_second': 1.193, 'epoch': 0.14}
{'loss': 2.4123, 'learning_rate': 9.504504504504504e-05, 'epoch': 0.15}
{'loss': 2.4034, 'learning_rate': 9.45945945945946e-05, 'epoch': 0.16}
{'loss': 2.3773, 'learning_rate': 9.414414414414416e-05, 'epoch': 0.18}
{'loss': 2.3189, 'learning_rate': 9.36936936936937e-05, 'epoch': 0.19}
{'loss': 2.333, 'learning_rate': 9.324324324324324e-05, 'epoch': 0.2}
{'loss': 2.2993, 'learning_rate': 9.279279279279279e-05, 'epoch': 0.22}
{'loss': 2.3056, 'learning_rate': 9.234234234234235e-05, 'epoch': 0.23}
{'loss': 2.2725, 'learning_rate': 9.18918918918919e-05, 'epoch': 0.24}
{'loss': 2.2531, 'learning_rate': 9.144144144144145e-05, 'epoch': 0.26}
{'loss': 2.2582, 'learning_rate': 9.0990990990991e-05, 'epoch': 0.27}


...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.7086883783340454, 'eval_runtime': 100.1237, 'eval_samples_per_second': 9.968, 'eval_steps_per_second': 1.248, 'epoch': 2.84}
{'loss': 1.7742, 'learning_rate': 4.954954954954955e-06, 'epoch': 2.85}
{'loss': 1.7731, 'learning_rate': 4.504504504504505e-06, 'epoch': 2.86}
{'loss': 1.774, 'learning_rate': 4.0540540540540545e-06, 'epoch': 2.88}
{'loss': 1.8027, 'learning_rate': 3.603603603603604e-06, 'epoch': 2.89}
{'loss': 1.7832, 'learning_rate': 3.153153153153153e-06, 'epoch': 2.9}
{'loss': 1.7615, 'learning_rate': 2.702702702702703e-06, 'epoch': 2.92}
{'loss': 1.766, 'learning_rate': 2.2522522522522524e-06, 'epoch': 2.93}
{'loss': 1.7645, 'learning_rate': 1.801801801801802e-06, 'epoch': 2.94}
{'loss': 1.7731, 'learning_rate': 1.3513513513513515e-06, 'epoch': 2.96}
{'loss': 1.7934, 'learning_rate': 9.00900900900901e-07, 'epoch': 2.97}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.708711862564087, 'eval_runtime': 101.33, 'eval_samples_per_second': 9.849, 'eval_steps_per_second': 1.234, 'epoch': 2.97}
{'loss': 1.7675, 'learning_rate': 4.504504504504505e-07, 'epoch': 2.99}
{'loss': 1.7676, 'learning_rate': 0.0, 'epoch': 3.0}
{'train_runtime': 28375.0387, 'train_samples_per_second': 2.004, 'train_steps_per_second': 0.016, 'train_loss': 1.9319579338168238, 'epoch': 3.0}


TrainOutput(global_step=444, training_loss=1.9319579338168238, metrics={'train_runtime': 28375.0387, 'train_samples_per_second': 2.004, 'train_steps_per_second': 0.016, 'train_loss': 1.9319579338168238, 'epoch': 3.0})

In [17]:
batch = tokenizer("### Caption: Eromame's art features a female Corrin and Incineroar in armor from Fire Emblem, with long hair, a hairband, pointy ears, and an open mouth, against a simple background, and posted on their Twitter.", return_tensors='pt').to("cuda")

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=300)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\torch\utils\checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\autograd\_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")




 ### Caption: Eromame's art features a female Corrin and Incineroar in armor from Fire Emblem, with long hair, a hairband, pointy ears, and an open mouth, against a simple background, and posted on their Twitter.
### Tags:1girl, armor, fire_emblem, fire_emblem_femme_corrin, incineroar, long_hair, pointy_ears, point_of_view, twitter,


In [15]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\panta\.cache\huggingface\token
Login successful


In [16]:
model.push_to_hub("ooferdoodles/text2tags-opt-1.3b", use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ooferdoodles/text2tags-opt-1.3b/commit/8f57a538dc7425fb94694cd2c5df16bf0f08eec8', commit_message='Upload model', commit_description='', oid='8f57a538dc7425fb94694cd2c5df16bf0f08eec8', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
model.save_pretrained(r"loras/tagger-v4")

In [ ]:
%load_ext tensorboard